In [35]:
import os
import pickle
import click
import mlflow
import numpy as np
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

In [36]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("random-forest-hyperopt")


<Experiment: artifact_location=('file:///C:/Users/SebastianSy/Desktop/231110_Backup Gateway '
 'Final/Administration/Other/Sinn/Start-up and Job/Programmieren/MLOps '
 'Zoomcamp/week 2/mlruns/4'), creation_time=1716815613222, experiment_id='4', last_update_time=1716815613222, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>

In [37]:
def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


In [38]:
def run_optimization(data_path: str, num_trials: int):

    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

    def objective(params):

        with mlflow.start_run():
            mlflow.log_param("max_depth", params["max_depth"])
            mlflow.log_param("min_samples_leaf", params["min_samples_leaf"])
            mlflow.log_param("min_samples_split", params["min_samples_split"])
            mlflow.log_param("n_estimators", params["n_estimators"])
            mlflow.log_param("random_state", params["random_state"])
            rf = RandomForestRegressor(**params)
            rf.fit(X_train, y_train)
            y_pred = rf.predict(X_val)
            rmse = root_mean_squared_error(y_val, y_pred)
            
            mlflow.log_metric("rmse", rmse)
            mlflow.end_run()
        return {'loss': rmse, 'status': STATUS_OK}

    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
        'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
        'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
        'random_state': 42
    }
    rstate = np.random.default_rng(42)  # for reproducible results
    fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=num_trials,
        trials=Trials(),
        rstate=rstate
    )


In [39]:
run_optimization(data_path="./output", num_trials=15)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [02:00<00:00,  8.05s/trial, best loss: 5.335419588556921]
